In [1]:
import src.config as config

In [2]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'{current_date=}')

# we fetch raw data for the last 28 days, to add redundancy to our data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=28)

current_date=Timestamp('2025-03-04 09:00:00')


In [3]:
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date=}, {to_date_=}')

    # download 2 files from website
    rides = load_raw_data(year=from_date_.year, months=from_date_.month)
    rides = rides[rides.pickup_datetime >= from_date_]
    rides_2 = load_raw_data(year=to_date_.year, months=to_date_.month)
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]

    rides = pd.concat([rides, rides_2])

    # shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [4]:
rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)

from_date=Timestamp('2025-02-04 09:00:00'), to_date_=Timestamp('2024-03-05 09:00:00')
File 2024_02 was already in local storage
File 2024_03 was already in local storage


In [5]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 265/265 [00:00<00:00, 1139.81it/s]


In [8]:
# string to datetime
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'], utc=True)

# add column with Unix epoch milliseconds
ts_data['pickup_ts'] = ts_data['pickup_hour'].astype(int) // 10**6

In [9]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_ts'],
    event_time='pickup_ts',
)

2025-03-04 20:37:14,257 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 20:37:14,262 INFO: Initializing external client
2025-03-04 20:37:14,262 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 20:37:16,158 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214654


In [10]:
feature_group.insert(ts_data, write_options={"wait_for_job": True})

Uploading Dataframe: 100.00% |██████████| Rows 178080/178080 | Elapsed Time: 00:23 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214654/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions
2025-03-04 20:38:01,750 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2025-03-04 20:38:05,044 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-03-04 20:38:08,359 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-03-04 20:41:15,961 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: UNDEFINED
2025-03-04 20:41:25,874 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-03-04 20:41:26,189 INFO: Waiting for log aggregation to finish.
2025-03-04 20:41:38,744 INFO: Execution finished successfully.


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)